In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))
import time

In [ ]:
import ee 
import geemap

ee.Initialize(project="thurgau-irrigation")

In [ ]:
from src.data_processing.downscaling import resample_image, Downscaler
from src.data_processing.sentinel_preprocessing import load_sentinel2_data

from utils.date_utils import (
    set_to_first_of_month,
    print_collection_dates,
    create_centered_date_ranges,
)
from utils.ee_utils import harmonized_ts, export_image_to_asset, back_to_int
from utils.harmonic_regressor import HarmonicRegressor
from utils.wapor_et_processing import load_wapor_et_data

from typing import List, Callable, Tuple

# 0: Define the region of interest

In [ ]:
PATH_TO_AOI = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"

aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

aoi = aoi_geometry.buffer(100)

# 1: Exporting dekdal WaPOR ET data for Thurgau:

In [ ]:
# first_year = 2023
# last_year = 2023
# wapor_et_data = load_wapor_et_data(
#     first_year, last_year, frequency="dekadal"
# ).filterBounds(aoi)



In [ ]:
# SCALE = wapor_et_data.first().projection().nominalScale().getInfo()

# print(f"The scale of the data is {SCALE}")

# for year in range(2023, 2024):
#     et_wapor_year = wapor_et_data.filterDate(f"{year}-01-01", f"{year}-12-31")

#     et_wapor_year_list = et_wapor_year.toList(et_wapor_year.size())

#     for i in range(36):
#         dekadal = i % 3 + 1
#         month = i // 3 + 1

#         image_to_export = ee.Image(et_wapor_year_list.get(i))

#         task_name = f"Thurgau_ET_WAPOR_300m_{year}_{month}_{dekadal}"
#         asset_id = f"projects/thurgau-irrigation/assets/ET_products/Thurgau/WaPOR_300m/{task_name}"

#         export_image_to_asset(
#             image=image_to_export,
#             asset_id=asset_id,
#             task_name=task_name,
#             year=year,
#             scale=SCALE,
#             aoi=aoi,
#         )

#         print(f"Exported image {task_name} to asset {asset_id}")

#     print(f"Done exporting year {year}, sleeping for 15 seconds")
#     time.sleep(15)

# print("Done exporting all images, mate!")

# 2: Exporting monthly landsat at 100m resolution

In [9]:
# TODO: Once Silvan is done exporting to that collection

# Sanity check

In [7]:
Map = geemap.Map()

image_collection = ee.ImageCollection("projects/thurgau-irrigation/assets/ET_products/Thurgau/WaPOR_300m")
image = ee.Image(image_collection.toList(image_collection.size()).get(20))

Map.addLayer(image, {"min": 0, "max": 5, "palette": ["blue", "yellow", "red"]}, "ET")

Map.centerObject(aoi, 10)

Map



Map(center=[47.56858787382066, 9.092720596553875], controls=(WidgetControl(options=['position', 'transparent_b…

In [8]:
print_collection_dates(image_collection)

Dates of images in the collection:
2018-10-01
2018-10-11
2018-10-21
2018-11-01
2018-11-11
2018-11-21
2018-12-01
2018-12-11
2018-12-21
2018-01-01
2018-01-11
2018-01-21
2018-02-01
2018-02-11
2018-02-21
2018-03-01
2018-03-11
2018-03-21
2018-04-01
2018-04-11
2018-04-21
2018-05-01
2018-05-11
2018-05-21
2018-06-01
2018-06-11
2018-06-21
2018-07-01
2018-07-11
2018-07-21
2018-08-01
2018-08-11
2018-08-21
2018-09-01
2018-09-11
2018-09-21
2019-10-01
2019-10-11
2019-10-21
2019-11-01
2019-11-11
2019-11-21
2019-12-01
2019-12-11
2019-12-21
2019-01-01
2019-01-11
2019-01-21
2019-02-01
2019-02-11
2019-02-21
2019-03-01
2019-03-11
2019-03-21
2019-04-01
2019-04-11
2019-04-21
2019-05-01
2019-05-11
2019-05-21
2019-06-01
2019-06-11
2019-06-21
2019-07-01
2019-07-11
2019-07-21
2019-08-01
2019-08-11
2019-08-21
2019-09-01
2019-09-11
2019-09-21
2020-10-01
2020-10-11
2020-10-21
2020-11-01
2020-11-11
2020-11-21
2020-12-01
2020-12-11
2020-12-21
2020-01-01
2020-01-11
2020-01-21
2020-02-01
2020-02-11
2020-02-21
2020-03-